In [1]:
import numpy as np
import cupy as cp
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from scipy import interp
from itertools import cycle
from tensorflow.keras import layers, models
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc


#  Data Processing

In [2]:
gear_defects = [0, 35, 76, 77]

In [3]:
# Input Image Size
size = [300, 400]

In [4]:
X_train = np.load('gears_train_300x400_0,76,77,35.npy')
X_test = np.load('gears_test_300x400_0,76,77,35.npy')
y_train = np.load('gears_ytrain_300x400_0,76,77,35.npy')
y_test = np.load('gears_ytest_300x400_0,76,77,35.npy')

# Training 

In [5]:
# Define early stopping and best model checkpoint
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, min_delta=0.1, patience=40)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_weights_only=True, mode='max', verbose=1, save_best_only=True)

In [6]:
dropout = 0.5
model = models.Sequential()
model.add(layers.Conv2D(64, (11, 11), strides=(4,4), activation='relu', input_shape=(size[0],size[1],1)))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Conv2D(128, (5, 5), strides=(1,1), padding="same", activation='tanh'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1), padding="same", activation='tanh'))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1), padding="same", activation='tanh'))
model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding="same", activation='tanh'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='tanh'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(256, activation='tanh'))
model.add(layers.Dense(len(np.unique(y_train))))
model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 73, 98, 64)        7808      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 36, 48, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 36, 48, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 36, 48, 128)       204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 17, 23, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 17, 23, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 23, 256)       2

In [7]:
# Fitting model with early stopping and checkpoint save
history = model.fit(X_train, y_train, epochs=100, 
                    validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/100
262/262 [==============================] - ETA: 0s - loss: 0.8061 - accuracy: 0.7798
Epoch 00001: val_accuracy improved from -inf to 0.75048, saving model to best_model.h5
262/262 [==============================] - 6s 25ms/step - loss: 0.8061 - accuracy: 0.7798 - val_loss: 0.9015 - val_accuracy: 0.7505
Epoch 2/100
261/262 [============================>.] - ETA: 0s - loss: 0.7496 - accuracy: 0.7880
Epoch 00002: val_accuracy did not improve from 0.75048
262/262 [==============================] - 6s 23ms/step - loss: 0.7488 - accuracy: 0.7883 - val_loss: 0.8568 - val_accuracy: 0.7505
Epoch 3/100
262/262 [==============================] - ETA: 0s - loss: 0.7421 - accuracy: 0.7883
Epoch 00003: val_accuracy did not improve from 0.75048
262/262 [==============================] - 6s 22ms/step - loss: 0.7421 - accuracy: 0.7883 - val_loss: 0.8569 - val_accuracy: 0.7505
Epoch 4/100
262/262 [==============================] - ETA: 0s - loss: 0.7463 - accuracy: 0.7883
Epoch 00004: val_ac

Epoch 30/100
262/262 [==============================] - ETA: 0s - loss: 0.7385 - accuracy: 0.7883
Epoch 00030: val_accuracy did not improve from 0.75048
262/262 [==============================] - 6s 22ms/step - loss: 0.7385 - accuracy: 0.7883 - val_loss: 0.8876 - val_accuracy: 0.7505
Epoch 31/100
262/262 [==============================] - ETA: 0s - loss: 0.7390 - accuracy: 0.7883
Epoch 00031: val_accuracy did not improve from 0.75048
262/262 [==============================] - 6s 22ms/step - loss: 0.7390 - accuracy: 0.7883 - val_loss: 0.8539 - val_accuracy: 0.7505
Epoch 32/100
262/262 [==============================] - ETA: 0s - loss: 0.7397 - accuracy: 0.7883
Epoch 00032: val_accuracy did not improve from 0.75048
262/262 [==============================] - 6s 22ms/step - loss: 0.7397 - accuracy: 0.7883 - val_loss: 0.8495 - val_accuracy: 0.7505
Epoch 33/100
262/262 [==============================] - ETA: 0s - loss: 0.7372 - accuracy: 0.7883
Epoch 00033: val_accuracy did not improve from 

In [8]:
model.load_weights('best_model.h5')

In [9]:
model.save('alexnet_tanh.h5')

In [10]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'alexnt_tanh_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [11]:
dropout = 0.5
model = models.Sequential()
model.add(layers.Conv2D(64, (11, 11), strides=(4,4), activation='relu', input_shape=(size[0],size[1],1)))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Conv2D(128, (5, 5), strides=(1,1), padding="same", activation='sigmoid'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1), padding="same", activation='sigmoid'))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1), padding="same", activation='sigmoid'))
model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding="same", activation='sigmoid'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='sigmoid'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(256, activation='sigmoid'))
model.add(layers.Dense(len(np.unique(y_train))))
model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 73, 98, 64)        7808      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 36, 48, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 36, 48, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 36, 48, 128)       204928    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 17, 23, 128)       0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 17, 23, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 17, 23, 256)      

In [12]:
# Define early stopping and best model checkpoint
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, min_delta=0.1, patience=40)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_weights_only=True, mode='max', verbose=1, save_best_only=True)

In [13]:
# Fitting model with early stopping and checkpoint save
history = model.fit(X_train, y_train, epochs=100, 
                    validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/100
262/262 [==============================] - ETA: 0s - loss: 0.7540 - accuracy: 0.7866
Epoch 00001: val_accuracy improved from -inf to 0.75048, saving model to best_model.h5
262/262 [==============================] - 6s 23ms/step - loss: 0.7540 - accuracy: 0.7866 - val_loss: 0.8956 - val_accuracy: 0.7505
Epoch 2/100
262/262 [==============================] - ETA: 0s - loss: 0.7467 - accuracy: 0.7883
Epoch 00002: val_accuracy did not improve from 0.75048
262/262 [==============================] - 6s 22ms/step - loss: 0.7467 - accuracy: 0.7883 - val_loss: 0.8734 - val_accuracy: 0.7505
Epoch 3/100
262/262 [==============================] - ETA: 0s - loss: 0.7468 - accuracy: 0.7883
Epoch 00003: val_accuracy did not improve from 0.75048
262/262 [==============================] - 6s 22ms/step - loss: 0.7468 - accuracy: 0.7883 - val_loss: 0.8558 - val_accuracy: 0.7505
Epoch 4/100
262/262 [==============================] - ETA: 0s - loss: 0.7401 - accuracy: 0.7883
Epoch 00004: val_ac

Epoch 30/100
262/262 [==============================] - ETA: 0s - loss: 0.7317 - accuracy: 0.7883
Epoch 00030: val_accuracy did not improve from 0.75048
262/262 [==============================] - 6s 22ms/step - loss: 0.7317 - accuracy: 0.7883 - val_loss: 0.8699 - val_accuracy: 0.7505
Epoch 31/100
262/262 [==============================] - ETA: 0s - loss: 0.7316 - accuracy: 0.7883
Epoch 00031: val_accuracy did not improve from 0.75048
262/262 [==============================] - 6s 22ms/step - loss: 0.7316 - accuracy: 0.7883 - val_loss: 0.8739 - val_accuracy: 0.7505
Epoch 32/100
262/262 [==============================] - ETA: 0s - loss: 0.7316 - accuracy: 0.7883
Epoch 00032: val_accuracy did not improve from 0.75048
262/262 [==============================] - 6s 22ms/step - loss: 0.7316 - accuracy: 0.7883 - val_loss: 0.8713 - val_accuracy: 0.7505
Epoch 33/100
262/262 [==============================] - ETA: 0s - loss: 0.7317 - accuracy: 0.7883
Epoch 00033: val_accuracy did not improve from 

In [14]:
model.load_weights('best_model.h5')

In [15]:
model.save('alexnet_sigmoid.h5')

In [16]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'alexnt_sigmoid_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)